### Load in Data and Libraries

In [200]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

In [202]:
house_data = pd.read_csv('data/kc_house_data.csv')
display(Markdown('data/column_names.md'))

# Column Names and Descriptions for King County Data Set
* `id` - Unique identifier for a house
* `date` - Date house was sold
* `price` - Sale price (prediction target)
* `bedrooms` - Number of bedrooms
* `bathrooms` - Number of bathrooms
* `sqft_living` - Square footage of living space in the home
* `sqft_lot` - Square footage of the lot
* `floors` - Number of floors (levels) in house
* `waterfront` - Whether the house is on a waterfront
  * Includes Duwamish, Elliott Bay, Puget Sound, Lake Union, Ship Canal, Lake Washington, Lake Sammamish, other lake, and river/slough waterfronts
* `view` - Quality of view from house
  * Includes views of Mt. Rainier, Olympics, Cascades, Territorial, Seattle Skyline, Puget Sound, Lake Washington, Lake Sammamish, small lake / river / creek, and other
* `condition` - How good the overall condition of the house is. Related to maintenance of house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each condition code
* `grade` - Overall grade of the house. Related to the construction and design of the house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each building grade code
* `sqft_above` - Square footage of house apart from basement
* `sqft_basement` - Square footage of the basement
* `yr_built` - Year when house was built
* `yr_renovated` - Year when house was renovated
* `zipcode` - ZIP Code used by the United States Postal Service
* `lat` - Latitude coordinate
* `long` - Longitude coordinate
* `sqft_living15` - The square footage of interior housing living space for the nearest 15 neighbors
* `sqft_lot15` - The square footage of the land lots of the nearest 15 neighbors


In [230]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

## Appropriating Data Types

In [269]:
house_data['date'] = pd.to_datetime(house_data['date'])

In [196]:
house_data['bathrooms'] = house_data['bathrooms'].astype(int)

In [ ]:
house_data['']

In [78]:
house_data['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
862.0         1
2120.0        1
2730.0        1
666.0         1
1816.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [86]:
#1.5% of sqft_basement has ? value
sum(house_data.loc[house_data['sqft_basement']== '?'].value_counts())/len(house_data) 
#59% of sqft of basements are 0.0 so will be replacing ? with 0.0
sum(house_data.loc[house_data['sqft_basement'] == '0.0']['sqft_basement'].value_counts())/len(house_data)


0.5938787794601103

In [250]:
house_data['sqft_basement'].replace(to_replace = '?', value='0.0', inplace = True)

In [251]:
house_data['sqft_basement'] = house_data['sqft_basement'].astype(float)

In [194]:
house_data['grade'].value_counts()

7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [221]:
house_data['grade'].replace(('3 Poor','4 Low','5 Fair', '6 Low Average','7 Average', '8 Good', '9 Better', '10 Very Good','11 Excellent', '12 Luxury', '13 Mansion'), ('0','1','2','3','4','5','6','7','8','9','10'), inplace =True)
house_data['grade'] = house_data['grade'].astype(int)

In [228]:
house_data['condition'].value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [229]:
house_data['condition'].replace(('Poor','Fair','Average','Good','Very Good'),('1','2','3','4','5'),inplace = True) 
house_data['condition']= house_data['condition'].astype(int)

In [231]:
house_data['condition'].value_counts()

3    14020
4     5677
5     1701
2      170
1       29
Name: condition, dtype: int64

In [233]:
house_data['view'].value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [236]:
house_data['view'].replace(('NONE','FAIR','AVERAGE','GOOD','EXCELLENT'),('1','2','3','4','5'),inplace = True)

In [256]:
house_data['waterfront'].isna().sum()

2376

In [260]:
house_data['waterfront'].replace(('NO','YES'),('0','1'),inplace=True)
house_data['waterfront'] = house_data['waterfront'].astype(int)

In [267]:
house_data['yr_renovated']= house_data['yr_renovated'].astype(int)

## NULL VALUES CLEANING

In [34]:
house_data.loc[house_data['waterfront'].isna()] 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
10,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,NaN,NONE,...,8 Good,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925
23,8091400200,5/16/2014,252700.0,2,1.50,1070,9643,1.0,NaN,NONE,...,7 Average,1070,0.0,1985,NaN,98030,47.3533,-122.166,1220,8386
40,5547700270,7/15/2014,625000.0,4,2.50,2570,5520,2.0,NaN,NONE,...,9 Better,2570,0.0,2000,NaN,98074,47.6145,-122.027,2470,5669
55,9822700295,5/12/2014,885000.0,4,2.50,2830,5000,2.0,NaN,NONE,...,9 Better,2830,0.0,1995,0.0,98105,47.6597,-122.290,1950,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21578,5087900040,10/17/2014,350000.0,4,2.75,2500,5995,2.0,NaN,NONE,...,8 Good,2500,0.0,2008,0.0,98042,47.3749,-122.107,2530,5988
21582,8956200760,10/13/2014,541800.0,4,2.50,3118,7866,2.0,NaN,AVERAGE,...,9 Better,3118,0.0,2014,0.0,98001,47.2931,-122.264,2673,6500
21586,844000965,6/26/2014,224000.0,3,1.75,1500,11968,1.0,NaN,NONE,...,6 Low Average,1500,0.0,2014,0.0,98010,47.3095,-122.002,1320,11303
21587,7852140040,8/25/2014,507250.0,3,2.50,2270,5536,2.0,NaN,NONE,...,8 Good,2270,0.0,2003,0.0,98065,47.5389,-121.881,2270,5731


In [102]:
house_data['waterfront'].value_counts()

NO     21451
YES      146
Name: waterfront, dtype: int64

In [101]:
house_data['waterfront'].fillna(value='NO', inplace = True)

In [112]:
house_data['yr_renovated'].fillna(value = house_data['yr_renovated'].median(),inplace=True)

In [273]:
house_data['yr_renovated'].value_counts()

0       20853
2014       73
2003       31
2013       31
2007       30
        ...  
1976        1
1953        1
1951        1
1946        1
1944        1
Name: yr_renovated, Length: 70, dtype: int64

In [245]:
house_data['view'].value_counts()

1.0    19422
3.0      957
4.0      508
2.0      330
5.0      317
0.0       63
Name: view, dtype: int64

In [166]:
#FILL IN ALL NULL VALUES WITH NONE

In [134]:
house_data['view'].replace(to_replace = 'NONE','AVERAGE','GOOD','FAIR','EXCELLENT'), value=('1.0','2.0','3.0','4.0','5.0'), inplace= True)


In [244]:
house_data['view']= house_data['view'].astype(float)

In [240]:
house_data['view'].fillna(value ='0.0',inplace=True)

In [178]:
house_data['view']= house_data['view'].astype(float)

In [259]:
house_data['waterfront'].fillna(value='NO',inplace =True)

In [264]:
house_data['yr_renovated'].fillna(value =0,inplace=True)

In [270]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     21597 non-null  int64         
 9   view           21597 non-null  float64       
 10  condition      21597 non-null  int64         
 11  grade          21597 non-null  int64         
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21597 non-null  float64       
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   2159

In [271]:
house_data.to_csv('data/cleaned.csv')

In [277]:
house_data['year_sold'] = house_data['date'].dt.year
house_data['month_sold'] = house_data['date'].dt.month

In [280]:
house_data.to_csv('data/cleaned.csv')